In [1]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset

In [8]:
# load multi30k
from datasets import load_dataset
multi30k = load_dataset("bentrevett/multi30k")
multi30k

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
model_name = 'TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,           
    bnb_4bit_quant_type="nf4",    
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_compute_dtype=torch.bfloat16, 
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, 
    device_map="auto",  
    trust_remote_code=True, 
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) 
tokenizer.pad_token = tokenizer.eos_token

In [11]:
# Setting arguments for low-rank adaptation 

model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05 
lora_rank = 32 

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [55]:
output_dir = "HamadJassem/tinyllama2k" # Model repo on your hugging face account where you want to save your model
per_device_train_batch_size = 3
gradient_accumulation_steps = 2  
optim = "paged_adamw_32bit" 
save_strategy="steps" 
save_steps = 10 
logging_steps = 10  
learning_rate = 2e-3  
max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 2000     # Number of training steps
warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to='none'
)

In [13]:
def formatted_train(input, response):
    return f"<|user|>\n{input}</s>\n<|assistant|>\n{response}</s>"

def prepare_train_data(data_id):
    data = load_dataset(data_id)
    data_df = pd.DataFrame(data['train'])
    data_df['formatted'] = data_df.apply(lambda x: formatted_train(x['en'], x['de']), axis=1)
    return data_df['formatted'].tolist()


In [15]:
train_data = prepare_train_data("bentrevett/multi30k")

In [17]:
data = Dataset.from_dict({"text": train_data})


In [56]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data,
    peft_config=peft_config,
    max_seq_length=500,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

In [57]:
trainer.train()

  0%|          | 0/2000 [00:00<?, ?it/s]

/home/hamad.alhammadi/.conda/envs/pytorch-intro/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.2449, 'grad_norm': 0.7951179146766663, 'learning_rate': 0.0003333333333333333, 'epoch': 0.0}


/home/hamad.alhammadi/.conda/envs/pytorch-intro/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.1993, 'grad_norm': 0.9199689626693726, 'learning_rate': 0.0006666666666666666, 'epoch': 0.0}


...

/home/hamad.alhammadi/.conda/envs/pytorch-intro/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.193, 'grad_norm': 1.1305638551712036, 'learning_rate': 1.3111633436779792e-07, 'epoch': 0.41}


/home/hamad.alhammadi/.conda/envs/pytorch-intro/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.0554, 'grad_norm': 1.6062161922454834, 'learning_rate': 0.0, 'epoch': 0.41}
{'train_runtime': 947.0058, 'train_samples_per_second': 12.672, 'train_steps_per_second': 2.112, 'train_loss': 1.2585658359527587, 'epoch': 0.41}


TrainOutput(global_step=2000, training_loss=1.2585658359527587, metrics={'train_runtime': 947.0058, 'train_samples_per_second': 12.672, 'train_steps_per_second': 2.112, 'train_loss': 1.2585658359527587, 'epoch': 0.41})

In [5]:
def formatted_prompt(question)-> str:
    return f"<|user|>\n{question}</s>\n<|assistant|>:"


In [6]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.5,
      max_new_tokens=20,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  output=tokenizer.decode(outputs[0], skip_special_tokens=True)
  #print(output)
  output_time = perf_counter() - start_time
  #print(f"Time taken for inference: {round(output_time,2)} seconds")
  return output

In [10]:
test_data = load_dataset("bentrevett/multi30k", split="test")
test_data_df = pd.DataFrame(test_data)


In [13]:
generate_response(test_data_df.iloc[16]['en'])

'<|user|>\nA blond holding hands with a guy in the sand. \n<|assistant|>:\nEin Blonder hält Händchen mit einem Mann im Sand herum. \n'

In [134]:
print(test_data_df.iloc[16]['de'])

Eine Blondine hält mit einem Mann im Sand Händchen.


Loading The model

In [3]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, 
    device_map="auto",  
    trust_remote_code=True, 
)

model_path = "HamadJassem/tinyllama2k"
peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")
model = peft_model.merge_and_unload()

/home/hamad.alhammadi/.conda/envs/pytorch-intro/lib/python3.9/site-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [14]:
# for each test data, generate response and save it in a list

responses = []
for i in range(0, len(test_data_df)):
    response = generate_response(test_data_df.iloc[i]['en'])
    responses.append(response)

In [16]:
# save the responses in a new dataframe
new_df = test_data_df.copy()
new_df['generated_de'] = responses
new_df.to_csv('generated_responses_tinyllama2k.csv', index=False)
